In [1]:
# Michael 2018/12 preprocess dataset and combine different data source
import os
import numpy as np
import csv
import pandas as pd

In [2]:
celebrity_files_fake = os.listdir("./fakeNewsDatasetsVeronica2017/celebrityDataset/fake")
celebrity_files_legit = os.listdir("./fakeNewsDatasetsVeronica2017/celebrityDataset/legit")

news_files_fake = os.listdir("./fakeNewsDatasetsVeronica2017/fakeNewsDataset/fake")
news_files_ligit = os.listdir("./fakeNewsDatasetsVeronica2017/fakeNewsDataset/legit")

In [ ]:
# generate celebrity.csv
content = [["head","content","class","source","file_name"]]
for file in celebrity_files_fake:
    with open(os.path.join("./fakeNewsDatasetsVeronica2017/celebrityDataset/fake",file),'r',encoding="utf-8") as f:
        tmp_content = f.readlines()
    tmp = ""
    for i,l in enumerate(tmp_content):
        if i == 0:
            head = l.replace("\n","")
        tmp += l
    tmp = tmp.replace("\n","")
    content.append([head,tmp,"fake","celebrity",file])
for file in celebrity_files_legit:
    with open(os.path.join("./fakeNewsDatasetsVeronica2017/celebrityDataset/legit",file),'r',encoding="utf-8") as f:
        tmp_content = f.readlines()
    tmp = ""
    for i,l in enumerate(tmp_content):
        if i == 0:
            head = l.replace("\n","")
        tmp += l
    tmp = tmp.replace("\n","")
    content.append([head,tmp,"legit","celebrity",file])

with open("celebrity.csv","w",encoding="utf-8",newline='') as f:
    csv.writer(f).writerows(content)

In [ ]:
# generate news.csv
content = [["head","content","class","source","file_name"]]
for file in news_files_fake:
    with open(os.path.join("./fakeNewsDatasetsVeronica2017/fakeNewsDataset/fake",file),'r',encoding="utf-8") as f:
        tmp_content = f.readlines()
    tmp = ""
    for i, l in enumerate(tmp_content):
        if i == 0:
            head = l.replace("\n","")
        tmp += l
    tmp = tmp.replace("\n","")
    content.append([head,tmp,"fake","news",file])
for file in news_files_ligit:
    with open(os.path.join("./fakeNewsDatasetsVeronica2017/fakeNewsDataset/legit",file),'r',encoding="utf-8") as f:
        tmp_content = f.readlines()
    tmp = ""
    for i, l in enumerate(tmp_content):
        if i == 0:
            head = l.replace("\n","")
        tmp += l
    tmp = tmp.replace("\n","")    
    content.append([head,tmp,"legit","news",file])
with open("news.csv","w",encoding="utf-8",newline='') as f:
    csv.writer(f).writerows(content)

In [ ]:
news = pd.read_csv('news.csv')
celebrity = pd.read_csv("celebrity.csv")

In [ ]:
news["class"].value_counts()

In [ ]:
celebrity["class"].value_counts()

# Combined Data

In [3]:
import gensim
import nltk
import csv
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from multiprocessing import cpu_count
from gensim.models.word2vec import Word2Vec

D:\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
def preprocessing(data_location,data_destination,content_col = 'content',label_col = 'class',feature_num = 50, training_epoch = 10):
    data_raw = pd.read_csv(data_location)
    data_raw_columns = data_raw.columns.values
    print('Total instances:',data_raw.shape[0])
    data_raw = data_raw.dropna(how='any')
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    doc_list = [] # list of tokenized sentences
    bad_index = [] # index that should be removed
    for index,doc in enumerate(data_raw[content_col]):
        sen_list = []
        try:
            for s in sent_tokenize(doc):
                filtered_s = [stemmer.stem(w) for w in word_tokenize(s) if not w in stop_words]
                sen_list.extend(filtered_s)
        except:
            print("Wrong doc: " , doc,"Index: " , index)
            bad_index.extend([index])
            continue
        doc_list.append(sen_list)
    model = Word2Vec(doc_list,size=feature_num,min_count=1,workers=cpu_count(),iter=training_epoch)
    doc_vec = []
    for index, sentences in enumerate(doc_list):
        for word in sentences: 
            word_vec = None
            word_count = 0
            try:
                if word_vec:
                    word_vec += model.wv[word]
                else:
                    word_vec = model.wv[word]
                word_count += 1
            except:
                print("Error key", word)
                continue
        word_vec = word_vec / word_count
        doc_vec.append(word_vec)
    head = ['feature' + str(i) for i in range(len(doc_vec[0]))]
    head.extend(['class'])
    labels = data_raw[label_col].reshape((-1,1))
    labels = np.delete(labels,bad_index,axis=0)
    data_raw = np.delete(np.array(data_raw),bad_index,axis=0)
    
    print('Bad Index Num:',len(bad_index),'\tBad Index:',bad_index)
    print('Final label Num:',labels.shape)
    print('Doc list Num',len(doc_list))
    print('Doc Vec Num',np.array(doc_vec).shape)
    print('data_raw shape:',data_raw.shape)

    processed_data_vec = np.append(np.array(doc_vec),labels,axis = 1)
    processed_data_vec = np.append(np.array(head).reshape(1,-1),processed_data_vec,axis=0)
    
    processed_data_raw = np.append(np.array(data_raw_columns).reshape((1,-1)),data_raw,axis = 0)
    processed_data = np.append(processed_data_raw,processed_data_vec,axis = 1)
    
    with open(data_destination,"w",encoding="utf-8",newline='') as f:
        csv.writer(f).writerows(processed_data)

In [ ]:
csv_file = './new_combination'
csv_file = './data_expandedfeatures'
csv_file = './data_Wang_expandedfeatures'
# use combined data | delete mixture label because of its small sample size
data_combined = pd.read_csv(csv_file + '.csv')
print(data_combined['label'].value_counts())
data_combined = data_combined.loc[data_combined['label'].isin(['real','fake'])]
min_num_for_group = data_combined.groupby('label').size().reset_index(name='counts')['counts'].min()
print("minimum group number",min_num_for_group)
data_combined_balance = data_combined.groupby('label').apply(lambda x:x.sample(min_num_for_group))
data_combined_balance = data_combined_balance.drop(['title','author','url'],axis=1)
print('shape',data_combined_balance.shape)
data_combined_balance.to_csv( csv_file + '_processed.csv',index=False)
print('After process label distribution:\n',data_combined_balance['label'].value_counts())

In [ ]:
source = csv_file + '_processed.csv'
dest = csv_file + '_balanced.csv'
preprocessing(data_location=source,data_destination=dest,label_col='label',content_col='text')
tmp_combination = pd.read_csv(dest)
tmp_combination = tmp_combination.drop(['text','label'],axis=1)
print('final document shape',tmp_combination.shape)
tmp_combination.to_csv(dest,index=False)

In [ ]:
source = csv_file + '_processed.csv'
dest = csv_file + '_balanced_20compress.csv'
preprocessing(data_location=source,data_destination=dest,label_col='label',content_col='text',feature_num=20)
tmp_combination = pd.read_csv(dest)
tmp_combination = tmp_combination.drop(['text','label'],axis=1)
print('final document shape',tmp_combination.shape)
tmp_combination.to_csv(dest,index=False)